We have a DataFrame containing information on the bike station locations and their average demand/supply value at the end of a week and at the end of a day. We can use maps to visualize how the demand and supply varies across London and get useful insights. 

# Initializing the base map 
We use Folium, a python library for mapping and visualizations. We first initialize a base map of London to use to later visualize the demand data. 

In [18]:
import folium
import pandas as pd
import os 
from folium.plugins import HeatMap,MarkerCluster
import math

mapDaily = folium.Map(location = [51.5073219, -0.1276474], tiles='cartodbdark_matter' , zoom_start = 13 )
mapDaily

Next lets load in our data with the estimated weekly and daily averages 

In [19]:
bikeStationsDemandDaily = pd.read_pickle(os.path.join(os.getcwd(),r'pickles\bikeStationsDemandDaily.pkl'))
bikeStationsDemandWeekly = pd.read_pickle(os.path.join(os.getcwd(),r'pickles\bikeStationsDemandWeekly.pkl'))

# Visualization with Cluster Mapping and HeatMaps 

We will use Foliums Cluster Mapping and HeatMapping plugins to help us visualize how the demand and supply changes on the map. 
We first define a function that seperates



In [20]:
def mapClustersAndHeatMap(map,bikeStations):
   
    
    fg_supply_clusters = folium.FeatureGroup(name = "Supply", show = True)
    fg_demand_clusters = folium.FeatureGroup(name = "Demand", show = False)
    fg_heat_map = folium.FeatureGroup(name= "HeatMap",show= True)
    marker_cluster = MarkerCluster().add_to(fg_demand_clusters)
    marker_cluster2 = MarkerCluster().add_to(fg_supply_clusters)
    demandList = [] 
    supplyList = [] 
    for x in bikeStations.index:
        demand = bikeStations['demand'][x]
        supply = False
        if demand>0:
            demand = math.floor(demand)
        else:
            demand = abs(math.ceil(demand))
            supply=True
        if supply:
            for _ in range(1,demand +1):
                folium.Marker(
                location=[bikeStations['lat'][x], bikeStations['lon'][x]],
                popup=bikeStations['id'][x],
                icon=folium.Icon(color='green', prefix='fa',icon='bicycle')
                ).add_to(marker_cluster2)
                demandList.append([bikeStations['lat'][x], bikeStations['lon'][x]])
            
        else:
            for _ in range(1,demand +1):
                folium.Marker(
                location=[bikeStations['lat'][x], bikeStations['lon'][x]],
                popup=bikeStations['id'][x],
                icon=folium.Icon(color='red', prefix='fa',icon='bicycle')
                ).add_to(marker_cluster)
                supplyList.append([bikeStations['lat'][x], bikeStations['lon'][x]])


    HeatMap(demandList,gradient={.6: 'blue', .98: 'red', 1: 'red'}).add_to(fg_heat_map)
    HeatMap(supplyList,gradient={.6: 'blue', .98: 'lime', 1: 'green'}).add_to(fg_heat_map)
    fg_demand_clusters.add_to(map)
    fg_supply_clusters.add_to(map)
    fg_heat_map.add_to(map)
    
    return map

This function takes a bikestation array with estimated demand and a map. It takes the integeral values of the demand and adds a marker to one of the three feature groups. The first feature group is a marker cluster for demand values. The second is a marker cluster for demand and supply values. The third contains two heatmaps, one for demand value and one for supply values. 

## Daily Demand Mapping
Lets apply the function to the average daily demand values and see the resulting map 

In [21]:
mapDaily = mapClustersAndHeatMap(mapDaily,bikeStationsDemandDaily)
mapDaily.add_child(folium.LayerControl())
mapDaily


At the top right corner you can activate/deacivate the different cluster maps for supply and demand as well as the heatmap. 
The Green in the heatmap signfies higher areas of demand and the red signifies higher areas of supply. You can zoom into the map to see the exact estimate of demand and supply at particular bike stations or zoom out to get a better macro view of the supply/demand trends. 


## Daily Demand Analysis 
We see that there seems to be more demand outside of the 'central' London, around Westminster in this case. And there is quite a large amount of corresponding supply near the center of London along the Thames. What this suggests is that there are more Bike trips into central London than there are Bike trips out of it. However the daily data is rather limited by the fewer number of bikes. Lets take at the average demand and supply at the end of the week to try and see if these trends still hold. 


## Weekly Demand Mapping

We apply the same logic to the weekly maps and observe the results 

In [22]:
mapWeekly = folium.Map(location = [51.5073219, -0.1276474], tiles='cartodbdark_matter' , zoom_start = 12 )
mapWeekly = mapClustersAndHeatMap(mapWeekly,bikeStationsDemandWeekly)
mapWeekly.add_child(folium.LayerControl())
mapWeekly

## Weekly Demand Analysis 

The weekly maps follow the same trend, but they give us much more information on spread of demand and supply. As can be seen the supply is still most heavily concentrated around the center of London, these central areas are Soho, Westminster and near Whitechapel/London Bridge. Unlike the previous daily values, that the demand is more spread out than suggested by the daily values. The highest concentration of demand is in the Bayswater and Hoxton areas. However we can observe demand in areas all around London, in particular, the demand seems to form a circle around the central areas of high supply as mentioned above. This supports the hypothesis that the bikes tend to move from outside London into the center. 